Axioms for context <a class="ProveItLink" href="_context_.ipynb">proveit.physics.quantum.QPE</a>
========

In [1]:
import proveit
# Automation is not needed when building axiom expressions:
proveit.defaults.automation = False # This will speed things up.
from proveit import Literal
from proveit._common_ import a, b, eps, k, l, n, t
from proveit.linalg import MatrixProd, ScalarProd, TensorExp 
from proveit.logic import Equals, Forall, InSet
from proveit.number import zero, one, two, e, i, pi, Div, Exp, Integers, Interval, IntervalCO, NaturalsPos 
from proveit.number import Add, Exp, Floor, frac, LessEq, Mod, ModAbs, Mult, Prod, subtract, Sum
from proveit.physics.quantum import Ket, Meas, RegisterBra, RegisterSU, QubitRegisterSpace
from proveit.physics.quantum.QPE._common_ import (alpha_l, b_, delta_, m_, n_, phase_,
                                                  phase_m_, psi_k, Psi_, t_, two_pow_t,
                                                  u_, U_ )
from proveit.physics.quantum.QPE.phaseEstOps import ModAdd, Pfail, Psuccess
from proveit.statistics import Prob
# the context is in the current directory:
context = proveit.Context('.') # adds context root to sys.path if necessary

In [2]:
%begin axioms

Defining axioms for context 'proveit.physics.quantum.QPE'
Subsequent end-of-cell assignments will define axioms
%end_axioms will finalize the definitions


Let $U$ be a unitary operator that acts on $n$ qubits, with $\lvert u\rangle$ as an eigenstate of $U$ with eigenvalue $e^{2 \pi i \varphi}$:

In [3]:
nInNatPos = InSet(n_, NaturalsPos)

nInNatPos: n in NaturalsPos

In [4]:
unitaryU = InSet(U_, RegisterSU(n_))

unitaryU: U in [SU](2^{n})

In [5]:
uKetRegister = InSet(Ket(u_), QubitRegisterSpace(n_))

uKetRegister: |u〉 in (Complexes^{2})^{otimes n}

In [6]:
phaseInInterval = InSet(phase_, IntervalCO(zero, one))

phaseInInterval: phase in [0,1)

In [7]:
eigenUu = Equals(MatrixProd(U_, Ket(u_)), ScalarProd(Exp(e, Mult(two, pi, i, phase_)), Ket(u_)))

eigenUu: (U . |u〉) = (e^{2 * pi * i * phase} * |u〉)

In [8]:
# t (represented by the Literal t_)
# denotes the number of Qbits in the input register
tInNaturalsPos = InSet(t_, NaturalsPos)

tInNaturalsPos: t in NaturalsPos

In [9]:
psiKetsInQRegSpace = Forall(k, InSet(Ket(psi_k), QubitRegisterSpace(k)), domain=NaturalsPos)

psiKetsInQRegSpace: forall_{k in NaturalsPos} (|psi_{k}〉 in (Complexes^{2})^{otimes k})

In [10]:
PsiKetInQRegSpace = InSet(Ket(Psi_), QubitRegisterSpace(t_))

PsiKetInQRegSpace: |Psi〉 in (Complexes^{2})^{otimes t}

In [11]:
# firstQPE = Circuit([[Input(ket0), Hgate, CTRL_DN, Output(Ket(psi_1))],
#                 [Input(Ket(u_)), MultiWire(n_), Gate(U_), Output(Ket(u_))]])

In [12]:
# midQPE = Forall(k, 
#                Circuit([[Input(Ket(psi_k)), MultiWire(k), PASS, WIRE_DN, PASS, PASS],
#                 [Input(ket0), Hgate, CTRL_DN, WIRE_LINK, MultiWire(Add(k, one)), Output(Ket(psi_next))],
#                 [Input(Ket(u_)), MultiWire(n_), Gate(U_pow_two_pow_k), Output(Ket(u_)), PASS, PASS]]),
#        domain=NaturalsPos)

In [13]:
# finalQPE = Circuit([[Input(Ket(psi_t)), MultiWire(t_), Gate(InverseFourierTransform(t_)), Output(Ket(Psi_))]])

In [14]:
# trying here to define Psi_1 (the result at the end of "stage 1")
# represented in Nielsen & Chuang's Ex 5.20 (pg 222)
Psi_1 = Literal(stringFormat='Psi_1', latexFormat=r'\Psi_{1}')
phi_ = Literal(stringFormat='phi', latexFormat=r'\varphi')
Psi_1_def = Equals(Psi_1,
                   Mult(frac(one, Exp(two, frac(t_,two))),
                        Sum(k, Mult(Exp(e, Mult(two, pi, i, phase_, k)), Ket(k)), domain=Interval(zero, subtract(Exp(two, t_), one)))))

Psi_1_def: Psi_1 = ((1 / 2^{t / 2}) * (Sum_{k=0}^{2^{t} - 1} (e^{2 * pi * i * phase * k} * |k〉)))

Let $\lvert \Psi \rangle$ be the outcome of the $t$-qubit register of ${\rm QPE}(U, t)$ acting on $\lvert u \rangle$, $m$ be a random variable representing the measurement of Psi with the register interpretted as an integer (via binary representation), and $\varphi_m = 2 \pi m/2^t$ be the random variable phase outcome of ${\rm QPE}(U, t)$:

In [15]:
mDef = Equals(m_, Meas(Ket(Psi_)))


mDef: m = MEAS(|Psi〉)

In [16]:
phase_m_def = Equals(phase_m_, frac(m_, two_pow_t))

phase_m_def: phase_m = (m / 2^{t})

Let $b$ be the value for $m$ that gives the closest $\varphi_m$ to $\varphi$ without exceeding it, and let $\delta$ be this difference:

In [17]:
bestDef = Equals(b_, Floor(Mult(phase_, two_pow_t)))

bestDef: b = floor(phase * 2^{t})

In [18]:
deltaDef = Equals(delta_, subtract(phase_, frac(b_, two_pow_t)))

deltaDef: delta = (phase - (b / 2^{t}))

The probability of success is defined as $\theta_m$ being within some epsilon of $\theta$: 

In [19]:
successDef = Forall(
        eps,
        Equals(Psuccess(eps),
               Prob(LessEq(ModAbs(subtract(m_, b_), two_pow_t), eps), m_)), 
        domain=NaturalsPos)

successDef: forall_{eps in NaturalsPos} ([Psuccess](eps) = Pr_{m}[|m - b|_{mod 2^{t}} <= eps])

In [20]:
failDef = Forall(
    eps,
    Equals(Pfail(eps), subtract(one, Psuccess(eps))),
    domain=NaturalsPos)

failDef: forall_{eps in NaturalsPos} ([Pfail](eps) = (1 - [Psuccess](eps)))

Let $\alpha_l$ be the amplitude of $\lvert \Psi \rangle$ for the $b \oplus l$ state, where $\oplus$ is defined as addition modulo $2^t$:

In [21]:
modAddDef = Forall(
    (a, b),
    Equals(ModAdd(a, b), Mod(Add(a, b), two_pow_t)),
    domain=Integers)

modAddDef: forall_{a in Integers, b in Integers} ((a MOD_ADD b) = ((a + b) mod  2^{t}))

In [22]:
alphaDef = Forall(
    l,
    Equals(alpha_l,
           MatrixProd(RegisterBra(ModAdd(b_, l), t_),
                      Ket(Psi_))),
    domain=Integers)

alphaDef: forall_{l in Integers} (alpha_{l} = {t}_〈b MOD_ADD l|Psi〉)

In [23]:
%end axioms

Axioms may be imported from autogenerated _axioms_.py
